In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

df = pd.read_csv("C:\\Users\\Leo\\code\\ch04_LoRA\\data\\review_data.csv", encoding="cp949")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=0, stratify=df['labels'])
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def preprocess(data):
    res =  tokenizer(data['text'], padding='max_length', truncation=True, max_length=64)
    return res

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

training_args = TrainingArguments(
    output_dir="./saved_models/basic_sentiment_model2",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,

    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,

    logging_dir="./logs",
    logging_steps=10,
    use_cpu = True
)

def compute_metrics(predict):
    preds = np.argmax(predict.predictions, axis=1)
    acc = (preds == predict.label_ids).mean()
    return {"accuracy": acc}
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [2]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.626847,0.500000
2,No log,0.401939,0.750000
3,0.421800,0.307591,1.000000


TrainOutput(global_step=12, training_loss=0.37307171275218326, metrics={'train_runtime': 18.3043, 'train_samples_per_second': 2.622, 'train_steps_per_second': 0.656, 'total_flos': 1578666332160.0, 'train_loss': 0.37307171275218326, 'epoch': 3.0})

In [ ]:
test_texts = ["이 제품 너무 좋아요!", "별로예요. 추천 안함."]
inputs = tokenizer(test_texts, padding=True, truncation=True, max_length=64, return_tensors="pt")
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    print("결과 :", predictions.tolist())  # 1은 긍정, 0은 부정

결과 : [1, 0]


In [4]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)  
trainer.train(resume_from_checkpoint="./saved_models/basic_sentiment_model2/checkpoint-12")  # XXX 부분에 체크포인트 번호 입력



Epoch,Training Loss,Validation Loss


TrainOutput(global_step=12, training_loss=0.0, metrics={'train_runtime': 0.1596, 'train_samples_per_second': 300.678, 'train_steps_per_second': 75.17, 'total_flos': 1578666332160.0, 'train_loss': 0.0, 'epoch': 3.0})

In [5]:
trainer.save_model("./saved_models/basic_sentiment_model2/final_model")
tokenizer.save_pretrained("./saved_models/basic_sentiment_model2/final_model")  

('./saved_models/basic_sentiment_model2/final_model\\tokenizer_config.json',
 './saved_models/basic_sentiment_model2/final_model\\special_tokens_map.json',
 './saved_models/basic_sentiment_model2/final_model\\vocab.txt',
 './saved_models/basic_sentiment_model2/final_model\\added_tokens.json',
 './saved_models/basic_sentiment_model2/final_model\\tokenizer.json')

In [6]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("./saved_models/basic_sentiment_model2/final_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_models/basic_sentiment_model2/final_model")


test_texts = ["이 제품 너무 좋아요!", "별로예요. 추천 안함."]
inputs = tokenizer(test_texts, padding=True, truncation=True, max_length=64,return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    
preds = torch.argmax(outputs.logits, dim=1)
print("결과 :", preds.tolist())  # 1은 긍정, 0은 부정

결과 : [1, 0]
